### Regresión lineal 
El Dataset implementado en cuestión es 'Tips', este contiene diferentes variables independientes que pueden ser utlizadas como varaibles predictoras, sin embargo, en este ejemplo intentaremos desarrollar un modelo lineal que solo se ajuste a una variable y que su aprendizaje sea en base a una correlación entre esta y su variable dependiente, varianzas iguales, distribuciones Gaussianas, etc.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from scipy.stats import pearsonr, zscore, kurtosis, norm, shapiro
from sklearn.linear_model import LinearRegression


df = sns.load_dataset("tips")
df

#### Análisis descriptivo
Si bien definos utilizar unicamente dos variables en el ejemplo, realizaremos una visualización múltiple de cada relación entre las variables numéricas y las variables categóricas, una vez viendo las distintas fiugras que se crearon podemos observar las siguientes características:
- Hay un crecimiento de la variable Y(tips) en los datos que tengan las etiquetas 'Dinner' y 'Male'
- Hay mayores frecuencias de la etiqueta 2, 'Male' y 'Dinner' en las variables 'size', 'sex' y 'sime'
- En la variable 'smoker' no se determina un comportamiento claro

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(14,12))
sns.scatterplot(df,x="total_bill",y="tip",hue="sex",ax=ax[0,0])
sns.scatterplot(df,x="total_bill",y="tip",hue="smoker",ax=ax[0,1])
sns.scatterplot(df,x="total_bill",y="tip",hue="day",ax=ax[1,0])
sns.scatterplot(df,x="total_bill",y="tip",hue="time",ax=ax[1,1])
ax[0,0].grid("on")
ax[0,1].grid("on")
ax[1,0].grid("on")
ax[1,1].grid("on")
plt.show()

#### Análisis exploratorio de datos

Es la descripción y el conocimiento profundo en un conjunto de datos, nos ayuda a descubrir y comprender información valiosa para tener en cuenta previamente a generar un modelo, ya mencionamos al análisis bivariado donde utlizábamos más de una variable, ahora tocará realizar un análisis univariado en las variables X(total_bill) y Y(tip) y deducir si se ajustarán al modelo elegido. Algunos puntos clave, sobretodo si trata de una implementación de regresión lineal, a examinar son:

Varianza: medida de variabilidad que se basa en dividir por el número de la muestra la sumatoria de los cuadrados entre las diferencias delos valores y la media, nos indica que tan dispersos están nuestros valores con respecto a la media.

Correlación(Pearson): valor estadístico que indica la fuerza y dirección de una asociación lineal entre dos variables númericas. 

Outliers: valores que destacan por su notoria distinción(mayor o menor) a los demás del conjunto. Existen diferentes formas de hallar este tipo de valores, pero la más confiable es específica son los Valores Z: estos vendrían a serlos valores de la muestra representando cierta desviación éstandar. Con esta transformación,determinamos que los Valores Z mayores a 3 se consideran atípicos.La razón del uso de este valor como umbral proviene de la regla empírica, según la cual los datos que se encuentran dentro de 3 desviaciones éstandar con respecto a la media representan el 99.7% de los datos.Así podemos concluir con bastante seguridad que los datos que caen por afuera de este umbral son atípicos, ya que son distintos al 99.7% de los datos.


In [ ]:
corr, _ = pearsonr(df["total_bill"],df["tip"])

var_x = df["total_bill"].var()
var_y = df["tip"].var()

df_zscore = df[["total_bill","tip"]]

df_zscore["total_bill_zscore"] = zscore(df["total_bill"]).abs()
df_zscore["tip_zscore"] = zscore(df["tip"]).abs()

outliers = df_zscore.loc[(df_zscore["total_bill_zscore"] > 3) | (df_zscore["tip_zscore"] > 3),["total_bill","tip"]]

curtosis_x = kurtosis(df["total_bill"])
curtosis_y = kurtosis(df["tip"])

print("--------------------------------------------------------------------------------------------------")
print(f"Correlación pearsonr: {round(corr,2)}")
print(f"Varianza variable X: {round(var_x,2)} | Varianza variable Y: {round(var_x,2)}")
print(f"Outliers: {outliers.shape[0]}")
print(f"Curtosis variable X: {round(curtosis_x,1)} | Curtosis variable Y: {round(curtosis_y,1)}")
print("--------------------------------------------------------------------------------------------------")

#### Pruebas de normalidad
La normalidad o distribución de Gauss es un punto clave estadístico en el análisis de datos y muchos modelos regresores requieren de este concepto debido a la ayuda que trae a la hora de estudiar los datos, algunas características y medidas de este tipo de dsitribuciones son una media y mediana similares o un coeficiente de cúrtosis(medida que analiza las colas de una distribución) cercano a 0, también existen los gráficos de cajas siendo más explicativos que los histogramas y otro tipo de métodos más indagatorios como los siguientes:

Shapiro-Wilk: Prueba que utiliza un gráfico conformado por los valores Z(eje X) y las variables originales del conjunto(eje Y). Los valores Z son elaborados mediante la media y la desviación éstandar, cada uno representa una determinada desviación éstandar con respecto a la media. Este gráfico permite visualizar la supuesta normalidad en los datos, detectar outliers y comparar los datos observados con los valores esperados bajo una distrbución normal.

Kolmogorov-Smirnov: Prueba que utiliza un gráfico conformado por los valores del conjunto(eje X) y la función de distriubción acumulada o CFD(eje Y). Mediante los valores Z o Estandarizados genera la probabilidad de que un valor aleatorio sea igual o menor a un valor dado. Se compara la CDF de los datos con la de una distribución teórica, la similitud entre ambas líneas puede concluir que los datos son consistente con la distrubción teórica.

Estas pruebas que determinan si una distribución es normal o no tienen sus propias hipótesis nulas(H0) que concluyen afirmativamente una distribución Guassiana e hipótesis alternativas(H1) que niegan lo anteriormente dicho, haciendo uso del Valor P para rechazar o aceptar alguna de estas. Por último, La prueba de Shapiro-Wilk es más confiable.

In [ ]:
df["total_bill_zscore"] = zscore(df["total_bill"])
df["tip_zscore"] = zscore(df["tip"])

cdf_total_bill = norm.cdf(df["total_bill"], df["total_bill"].mean(), df["total_bill"].std()) 
cdf_tip = norm.cdf(df["tip"], df["tip"].mean(), df["tip"].std()) 

fig = make_subplots(rows=2, cols=2)

fig.add_trace(go.Scatter(x=df["total_bill_zscore"], y=df["total_bill"], mode="markers", name="Cuentas totales"), row=1,col=1)
fig.add_trace(go.Scatter(x=df["tip_zscore"], y=df["tip"], mode="markers", name="Propinas"), row=1,col=2)
fig.add_trace(go.Scatter(x=df["total_bill"].values, y=cdf_total_bill, mode="markers", name="Cuentas totales"), row=2,col=1)
fig.add_trace(go.Scatter(x=df["tip"].values, y=cdf_tip, mode="markers", name="Propinas"), row=2,col=2)
fig.update_layout(height=600, width=850, title_text="Shapiro-wilk & Kolmogorov-smirnov")

_, p_value_var_x = shapiro(df["total_bill"])
_, p_value_var_y = shapiro(df["tip"])

fig

#### Modelo lineal

Luego de haber obtenido los distintos resultados como una correlación igual 0.68, varianzas desiguales y una distribución sesgada positivamente se procede a generar el modelo lineal y ponerlo en práctica

In [19]:
linear_regression = LinearRegression()
 
total_bill = df["total_bill"].values.reshape((-1,1))

model = linear_regression.fit(total_bill,df["tip"])

# ------- regresión de nuevos datos -------

objects = np.array([[28.15],[12.5],[3.8],[8.25],[19.5],[32.7],[40.9],[45]])

predicts = model.predict(objects)

#### Dashboard que refleja el EDA desarrollado previo a la creación del modelo y su ajuste sobre los datos

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e2_body",children=[
    html.H1("Propinas",id="titulo",className="e2_title"),
    html.Div(id="dashboard",className="e2_dashboard",children=[
        html.Div(id="column-1",className="e2_column_1",children=[
            dcc.Dropdown(id="dropdown",className="e2_dropdown",
                        options=[
                            {"label":"Cuentas totales","value":"total_bill"},
                            {"label":"Propinas","value":"tip"}
                        ],
                        value="total_bill",
                        multi=False,
                        clearable=False),
            html.Div(className="e2_div_graphs",children=[
                dcc.Graph(id="graph-1",className="e2_graphs",figure={}), 
                dcc.Graph(id="graph-2",className="e2_graphs",figure={})
            ])
        ]),
        html.Div(id="column-2",className="e2_column_2",children=[
            html.H2("Shapiro-Wilk Valor-P",id="p_values_title",className="e2_p_values_title"),
            html.Div(id="p_values",className="e2_stats_div",children=[
                html.Div(id="p_value_var_x",className="e2_stats",children=[html.P(f"Variable X: {round(p_value_var_x,2)}",style={"font-size":"1em"})]),
                html.Div(id="p_value_var_y",className="e2_stats",children=[html.P(f"Variable Y: {round(p_value_var_y,2)}",style={"font-size":"0.98em"})])
            ]),
            html.Div(f"Correlación: {round(corr,2)}",className="e2_corr",id="corr"),
            dcc.Graph(id="graph-3",className="e2_graph_3",figure={})
        ]),
        html.P("Explicación: El ejemplo que se elaboró demuestra su simplicidad, su tamaño es diminuto y no hay más variables para relacionar linealmente ni analizar su distribución, sin embargo si sirve para ser utilizado como una descripción clara de un concepto muy presente. Aunque parezca lógico y simple a primera vista que mientras mayor sea el pago total(variable X) mayor sea la propina(variable Y), esta premonición puede llegar a traer deducciones erróneas, si bien la relación que generan las variables mencionadas es dinámica también he vistos datos particulares que se alejan bastante del patrón lineal, esto debido a que no es solo una cuestión de causa y efecto, sino que también están influenciadas por otras variables que pueden no ser inmediatamente evidentes. Ya fueron visualizadas algunas categorías que denotan su participación en la variabilidad, por ejemplo: los fines de semanas o en el horario de cena las propinas suelen ser mayores. Por ello, a diferencia del ruido, la distorsión generada por el sesgo tiene un origen explicativo y es sistemática.", className="e2_p")
    ])
])

@app.callback(
    [Output(component_id="graph-1",component_property="figure"),
    Output(component_id="graph-2",component_property="figure"),
    Output(component_id="graph-3",component_property="figure")],
    [Input(component_id="dropdown",component_property="value")]
)

def update_dash(slct_var):
    
    mean = df[slct_var].mean()
    median = df[slct_var].median()
    
    extr_list = [0]
    
    var_title = "Cuentas totales"
    
    if slct_var == "tip":
        extr_list.append(60)
        var_title = "Propinas"
    elif slct_var == "total_bill":
        extr_list.append(40)
        var_title = "Cuentas totales"
    
    histplot = go.Figure(go.Histogram(x=df[slct_var],name="Distribución"))
    histplot.add_trace(go.Scatter(x=[mean,mean],y=extr_list,mode="lines+markers",marker_color="red",name="Media"))
    histplot.add_trace(go.Scatter(x=[median,median],y=extr_list,mode="lines+markers",marker_color="green",name="Mediana"))
    histplot.update_layout(title="Histograma",xaxis_title=var_title)
    
    boxplot = px.box(df, y=slct_var, points="all", title="Gráfico de caja")
    
    scatter = go.Figure()
    scatter.add_trace(go.Scatter(x=df["total_bill"],y=df["tip"],mode="markers",marker_color="blue",name="Propinas reales"))
    scatter.add_trace(go.Scatter(x=objects.reshape(-1),y=predicts,mode="lines+markers",marker_color="red",name="Predicciones"))
    scatter.update_layout(title="Regresión Lineal",xaxis_title="Cuentas totales",yaxis_title="Propinas")

    return histplot, boxplot, scatter

if __name__ == "__main__":
    app.run_server(debug=False)

#### Error de bías
También denominado sesgo, es la diferencia entre la suposiciones asumidas por el modelo y los valores verdaderos. Sucede principalmente en algoritmos paramétricos que requieren de ciertas estructuras o relaciones en los datos para una mejor interpretación. Alto bías: requiere más suposiciones a la hora de estimar la función objetivo, ejemplos de algoritmos con alto bías: Regresión Lineal, Regresión Logística, Naive Bayes, algoritmos de series temporales.